# R1: Clinical Utility - Dynamic Risk Updating

## Reviewer Question

**Referee #1**: "What is the clinical utility of this model? How would it be used in practice?"

## Why This Matters

Demonstrating clinical utility is essential for:
- Showing how the model would be used in real-world clinical practice
- Understanding the value of updating predictions over time
- Validating that dynamic risk assessment improves long-term predictions

## Our Approach

We evaluate **dynamic risk updating** - a clinically realistic scenario where:

1. **Annual Updates**: Patients are seen annually, and risk predictions are updated each year
2. **Rolling Predictions**: At each visit, we use the model trained with data up to that point
3. **10-Year Risk Interpolation**: We compute cumulative 10-year risk using updated predictions
4. **Comparison**: We compare dynamic (updated annually) vs. static (enrollment only) predictions

**Clinical Scenario**: This mirrors real-world practice where:
- Patients have annual checkups
- Risk assessments are updated based on new information
- Long-term risk is estimated using the most current predictions

**Note**: This analysis uses age_offset pi batches, which represent predictions made at enrollment + 0, 1, 2, ..., 9 years. Each year's prediction uses a model trained with data up to that point.

## Key Findings

✅ **Dynamic risk updating improves discrimination** for 10-year risk prediction
✅ **Annual updates capture evolving risk factors** and disease progression
✅ **Clinically realistic approach** mirrors real-world practice
⚠️ **Limitation**: Not a fully prospective evaluation (some temporal leakage)


## 1. Load Age Offset Predictions

We use pi batches from age_offset analysis, which represent predictions made at different time points after enrollment.

In [1]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
import sys

sys.path.append('/Users/sarahurbut/aladynoulli2/pyScripts')
from evaluatetdccode import evaluate_major_diseases_wsex_with_bootstrap_dynamic_rolling

# Load data
base_path = Path('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/')
Y = torch.load(base_path / 'Y_tensor.pt', weights_only=False)
E = torch.load(base_path / 'E_matrix.pt', weights_only=False)
essentials = torch.load(base_path / 'model_essentials.pt', weights_only=False)

# Load pce_df
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
readRDS = robjects.r['readRDS']
pce_data = readRDS('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/pce_df_prevent.rds')
pce_df = pandas2ri.rpy2py(pce_data)

# Subset to batch 0-10000 (matching age_offset analysis)
start_idx, end_idx = 0, 10000
indices = list(range(start_idx, end_idx))
Y_batch = Y[indices]
E_batch = E[indices]
pce_df_batch = pce_df.iloc[indices].reset_index(drop=True)
disease_names = essentials['disease_names']

print("="*80)
print("LOADING AGE OFFSET PI BATCHES")
print("="*80)
print(f"Batch: {start_idx}-{end_idx}")
print(f"Y shape: {Y_batch.shape}")
print(f"pce_df shape: {pce_df_batch.shape}")

LOADING AGE OFFSET PI BATCHES
Batch: 0-10000
Y shape: torch.Size([10000, 348, 52])
pce_df shape: (10000, 16)


In [2]:
# Load age_offset pi batches (offsets 0-9)
pi_base_dir = Path('/Users/sarahurbut/Library/CloudStorage/Dropbox/age_offset_files')
pi_batches = []

for k in range(10):
    pi_filename = f'pi_enroll_fixedphi_age_offset_{k}_sex_{start_idx}_{end_idx}_try2_withpcs_newrun.pt'
    pi_path = pi_base_dir / pi_filename
    
    if not pi_path.exists():
        print(f"⚠️  File not found: {pi_path}")
        break
    
    pi_batch = torch.load(pi_path, weights_only=False)
    pi_batches.append(pi_batch)
    print(f"  Loaded offset {k}: {pi_filename} (shape: {pi_batch.shape})")

print(f"\n✓ Loaded {len(pi_batches)} pi batches")

  Loaded offset 0: pi_enroll_fixedphi_age_offset_0_sex_0_10000_try2_withpcs_newrun.pt (shape: torch.Size([10000, 348, 52]))
  Loaded offset 1: pi_enroll_fixedphi_age_offset_1_sex_0_10000_try2_withpcs_newrun.pt (shape: torch.Size([10000, 348, 52]))
  Loaded offset 2: pi_enroll_fixedphi_age_offset_2_sex_0_10000_try2_withpcs_newrun.pt (shape: torch.Size([10000, 348, 52]))
  Loaded offset 3: pi_enroll_fixedphi_age_offset_3_sex_0_10000_try2_withpcs_newrun.pt (shape: torch.Size([10000, 348, 52]))
  Loaded offset 4: pi_enroll_fixedphi_age_offset_4_sex_0_10000_try2_withpcs_newrun.pt (shape: torch.Size([10000, 348, 52]))
  Loaded offset 5: pi_enroll_fixedphi_age_offset_5_sex_0_10000_try2_withpcs_newrun.pt (shape: torch.Size([10000, 348, 52]))
  Loaded offset 6: pi_enroll_fixedphi_age_offset_6_sex_0_10000_try2_withpcs_newrun.pt (shape: torch.Size([10000, 348, 52]))
  Loaded offset 7: pi_enroll_fixedphi_age_offset_7_sex_0_10000_try2_withpcs_newrun.pt (shape: torch.Size([10000, 348, 52]))
  Loaded

## 2. Evaluate Dynamic Risk Updating (Rolling)

We evaluate 10-year risk prediction using rolling updates: at each year after enrollment, we use the prediction from the model trained for that offset.

In [3]:
print("="*80)
print("EVALUATING DYNAMIC RISK UPDATING (ROLLING)")
print("="*80)
print("\nThis evaluates 10-year risk using predictions updated annually.")
print("At year k after enrollment, we use predictions from offset k model.\n")

results_rolling = evaluate_major_diseases_wsex_with_bootstrap_dynamic_rolling(
    pi_batches=pi_batches,
    Y_100k=Y_batch,
    E_100k=E_batch,
    disease_names=disease_names,
    pce_df=pce_df_batch,
    n_bootstraps=100,
    follow_up_duration_years=10
)

results_rolling_df = pd.DataFrame(results_rolling).T.reset_index().rename(columns={'index': 'Disease'})
results_rolling_df['Method'] = 'Dynamic_Rolling'

print("\n" + "="*80)
print("DYNAMIC ROLLING RESULTS")
print("="*80)
display(results_rolling_df.sort_values('auc', ascending=False).head(15))

EVALUATING DYNAMIC RISK UPDATING (ROLLING)

This evaluates 10-year risk using predictions updated annually.
At year k after enrollment, we use predictions from offset k model.


Evaluating ASCVD (Dynamic 10-Year Risk, Rolling)...
AUC: 0.854 (0.839-0.869) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 831 (8.3%) (from 10000 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk, Rolling)...
AUC: 0.750 (0.729-0.772) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 581 (5.8%) (from 10000 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk, Rolling)...
AUC: 0.751 (0.724-0.775) (calculated on 9864 individuals)
Events (10-Year in Eval Cohort): 376 (3.8%) (from 9864 individuals)
Excluded 136 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk, Rolling)...
AUC: 0.743 (0.710-0.775) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 207 (2.

,Disease,auc,n_events,event_rate,ci_lower,ci_upper,Method
15,Bladder_Cancer,0.887065,49.0,0.491179,0.836115,0.934406,Dynamic_Rolling
0,ASCVD,0.853513,831.0,8.310000,0.838906,0.868843,Dynamic_Rolling
25,Parkinsons,0.810969,46.0,0.460138,0.739768,0.866694,Dynamic_Rolling
13,Prostate_Cancer,0.810581,204.0,4.486475,0.780032,0.846218,Dynamic_Rolling
11,Colorectal_Cancer,0.807915,105.0,1.050000,0.749462,0.865768,Dynamic_Rolling
6,Heart_Failure,0.798071,205.0,2.050000,0.752126,0.831018,Dynamic_Rolling
20,Rheumatoid_Arthritis,0.774273,123.0,1.234568,0.732458,0.807794,Dynamic_Rolling
7,Pneumonia,0.763393,335.0,3.350000,0.730770,0.791930,Dynamic_Rolling
14,Lung_Cancer,0.757874,75.0,0.750600,0.705979,0.814739,Dynamic_Rolling
23,Crohns_Disease,0.756259,31.0,0.311026,0.672085,0.833504,Dynamic_Rolling


## 3. Compare to Static Prediction (Enrollment Only)

For comparison, we also evaluate static 10-year risk using only the enrollment prediction (offset 0).

In [6]:
from evaluatetdccode import evaluate_major_diseases_wsex_with_bootstrap_dynamic

# For static prediction, we need to create a model-like object or use the from_pi version
# Actually, let's use the static 10-year results if available, or compute from offset 0 only

# Use offset 0 pi batch as static prediction
pi_static = pi_batches[0]  # Enrollment only

# We can use evaluate_major_diseases_wsex_with_bootstrap_dynamic_from_pi if available
# Or load static 10-year results from time_horizons analysis

static_results_path = Path('../../results/time_horizons/pooled_retrospective/static_10yr_results.csv')
if static_results_path.exists():
    static_results_df = pd.read_csv(static_results_path)
    static_results_df['Method'] = 'Static_Enrollment'
    print("="*80)
    print("STATIC 10-YEAR RESULTS (ENROLLMENT ONLY)")
    print("="*80)
    display(static_results_df.head(15))
else:
    print("⚠️  Static results file not found. Would need to compute from pi_static.")

STATIC 10-YEAR RESULTS (ENROLLMENT ONLY)


,Disease,AUC,CI_lower,CI_upper,N_Events,Event_Rate,Method
0,ASCVD,0.737116,0.734335,0.739766,34705,8.676250,Static_Enrollment
1,Parkinsons,0.721991,0.712265,0.732132,1839,0.459750,Static_Enrollment
2,Atrial_Fib,0.707440,0.703783,0.711135,15278,3.819500,Static_Enrollment
3,Bladder_Cancer,0.706332,0.694454,0.715048,2158,0.539500,Static_Enrollment
4,CKD,0.704942,0.700371,0.709660,8980,2.245000,Static_Enrollment
5,Heart_Failure,0.702035,0.697396,0.706287,8212,2.053000,Static_Enrollment
6,Prostate_Cancer,0.684406,0.679874,0.689727,7565,4.144252,Static_Enrollment
7,Stroke,0.681151,0.674489,0.686286,5686,1.421500,Static_Enrollment
8,Osteoporosis,0.676327,0.671509,0.681366,9145,2.286250,Static_Enrollment
9,All_Cancers,0.671052,0.667186,0.674603,20338,5.084500,Static_Enrollment


## 4. Comparison: Dynamic vs. Static

Compare discrimination (AUC) between dynamic rolling updates and static enrollment-only predictions.

In [19]:
if 'static_results_df' in locals():
    # Merge results for comparison
    # Note: static_results_df uses 'AUC', 'CI_lower', 'CI_upper' (uppercase)
    # results_rolling_df uses 'auc', 'ci_lower', 'ci_upper' (lowercase)
    # Rename columns to match before merging so suffixes work correctly
    rolling_for_merge = results_rolling_df[['Disease', 'auc', 'ci_lower', 'ci_upper']].copy()
    rolling_for_merge.columns = ['Disease', 'AUC', 'CI_lower', 'CI_upper']
    
    static_for_merge = static_results_df[['Disease', 'AUC', 'CI_lower', 'CI_upper']].copy()
    
    comparison = rolling_for_merge.merge(
        static_for_merge,
        on='Disease',
        suffixes=('_Rolling', '_Static')
    )
    
    # After merge, columns are: AUC_Rolling, AUC_Static, etc.
    comparison['AUC_Improvement'] = comparison['AUC_Rolling'] - comparison['AUC_Static']
    comparison = comparison.sort_values('AUC_Improvement', ascending=False)
    
    print("="*80)
    print("COMPARISON: DYNAMIC ROLLING vs STATIC ENROLLMENT")
    print("="*80)
    print("\nDiseases with largest improvement from annual updates:\n")
    
    # Create display dataframe with correct column names
    comparison_display = comparison[['Disease', 'AUC_Static', 'AUC_Rolling', 'AUC_Improvement']].copy()
    comparison_display.columns = ['Disease', 'Static AUC', 'Rolling AUC', 'Improvement']
    comparison_display['Static AUC'] = comparison_display['Static AUC'].apply(lambda x: f"{x:.3f}")
    comparison_display['Rolling AUC'] = comparison_display['Rolling AUC'].apply(lambda x: f"{x:.3f}")
    comparison_display['Improvement'] = comparison_display['Improvement'].apply(lambda x: f"{x:+.3f}")
    
    display(comparison_display.head(15))
    
    # Summary statistics
    print("\n" + "="*80)
    print("SUMMARY STATISTICS")
    print("="*80)
    print(f"Mean AUC improvement: {comparison['AUC_Improvement'].mean():.3f}")
    print(f"Median AUC improvement: {comparison['AUC_Improvement'].median():.3f}")
    print(f"Diseases with improvement: {(comparison['AUC_Improvement'] > 0).sum()} / {len(comparison)}")
    


COMPARISON: DYNAMIC ROLLING vs STATIC ENROLLMENT

Diseases with largest improvement from annual updates:



,Disease,Static AUC,Rolling AUC,Improvement
19,Bipolar_Disorder,0.494,0.692,+0.198
12,Breast_Cancer,0.557,0.743,+0.186
15,Bladder_Cancer,0.706,0.887,+0.181
23,Crohns_Disease,0.588,0.756,+0.168
22,Ulcerative_Colitis,0.590,0.755,+0.165
20,Rheumatoid_Arthritis,0.610,0.774,+0.164
11,Colorectal_Cancer,0.646,0.808,+0.162
17,Depression,0.491,0.620,+0.129
13,Prostate_Cancer,0.684,0.811,+0.126
7,Pneumonia,0.645,0.763,+0.119



SUMMARY STATISTICS
Mean AUC improvement: 0.103
Median AUC improvement: 0.103
Diseases with improvement: 27 / 28


## 5. Summary and Response

### Key Findings

1. **Dynamic risk updating improves discrimination**: Annual updates improve 10-year risk prediction compared to static enrollment-only predictions.

2. **Clinically realistic approach**: This mirrors real-world practice where patients are seen annually and risk assessments are updated.

3. **Captures evolving risk**: Annual updates allow the model to incorporate new information about disease progression and risk factor changes.

### Clinical Interpretation

**Static Prediction (Enrollment Only)**:
- Single risk assessment at enrollment
- Does not incorporate new information
- May become less accurate over time

**Dynamic Prediction (Annual Updates)**:
- Risk assessment updated annually
- Incorporates new clinical information
- Better reflects evolving patient risk

### Response to Reviewer

We demonstrate clinical utility through **dynamic risk updating**:

- **Annual risk updates**: Patients are seen annually, and predictions are updated using models trained with data up to that point
- **Improved discrimination**: Dynamic updates improve 10-year risk prediction compared to static enrollment-only predictions
- **Clinically realistic**: This approach mirrors real-world practice where risk assessments evolve with new information

**Limitation**: This analysis is not a fully prospective evaluation, as each year's prediction uses a model trained with data up to that point (some temporal leakage). However, it demonstrates the clinical value of updating predictions over time.

---